In [ ]:
pip install chess.com

In [13]:
from chessdotcom import get_player_profile
import chessdotcom
import json
import pandas as pd
import datetime
import time
import requests

In [9]:
dir(chessdotcom)

['ChessDotComError',
 'ChessDotComResponse',
 'Client',
 'RateLimitHandler',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'client',
 'get_club_details',
 'get_club_matches',
 'get_club_members',
 'get_country_clubs',
 'get_country_details',
 'get_country_players',
 'get_current_daily_puzzle',
 'get_leaderboards',
 'get_player_clubs',
 'get_player_current_games',
 'get_player_current_games_to_move',
 'get_player_game_archives',
 'get_player_games_by_month',
 'get_player_games_by_month_pgn',
 'get_player_profile',
 'get_player_stats',
 'get_player_team_matches',
 'get_player_tournaments',
 'get_random_daily_puzzle',
 'get_streamers',
 'get_team_match',
 'get_team_match_board',
 'get_team_match_live',
 'get_team_match_live_board',
 'get_titled_players',
 'get_tournament_details',
 'get_tournament_round',
 'get_tournament_round_group_details',
 'types',
 'utils']

In [19]:
chess_streamers = chessdotcom.get_streamers().streamers

In [20]:
len(chess_streamers)

463

In [44]:
GM_players = chessdotcom.get_titled_players('GM').players

In [51]:
chessdotcom.get_player_stats('magnescarlsen').json

{'stats': {'chess_rapid': {'last': {'rating': 610,
    'date': 1686889087,
    'rd': 84},
   'best': {'rating': 897,
    'date': 1686846094,
    'game': 'https://www.chess.com/game/live/80587198239'},
   'record': {'win': 5, 'loss': 12, 'draw': 0}},
  'chess_bullet': {'last': {'rating': 462, 'date': 1686848008, 'rd': 207},
   'record': {'win': 0, 'loss': 2, 'draw': 0}},
  'chess_blitz': {'last': {'rating': 630, 'date': 1505917634, 'rd': 290},
   'record': {'win': 0, 'loss': 1, 'draw': 0}},
  'fide': 0,
  'tactics': {'highest': {'rating': 437, 'date': 1686861510},
   'lowest': {'rating': 387, 'date': 1686861429}},
  'puzzle_rush': {}}}

In [91]:
all_players_stat = []
n=1
modes = ['rapid', 'blitz', 'bullet']
for player in GM_players:
    player_stat = {}
    print(n,end=' ')
    n+=1
    stat = chessdotcom.get_player_stats(player).json['stats']
    profile = chessdotcom.get_player_profile(player).json['player']
    
    player_stat['name'] = profile.get('name',None)
    player_stat['username'] = profile.get('username',None)
    player_stat['profile'] = profile.get('url',None)
    player_stat['user_id'] = profile.get('player_id',None)
    player_stat['title'] = profile.get('title',None)
    player_stat['fide'] = stat.get('fide',None)
    try:
        player_stat['country'] = requests.get(profile.get('country',None)).json()['name']   
    except:
        player_stat['country'] = None
        
    player_stat['followers'] = profile.get('followers',None)
    try:
        player_stat['joined'] = time.strftime("%d/%m/%Y", time.localtime(int(profile['joined'])))
    except:
        player_stat['joined'] = None
    try:
        player_stat['last_online'] = time.strftime("%d/%m/%Y", time.localtime(int(profile['last_online'])))
    except:
        player_stat['last_online'] = None
                         
    
    for mode in modes:                    
        player_stat[f'current_{mode}_rating'] = stat.get(f'chess_{mode}',{}).get('last',{}).get('rating', None)
        player_stat[f'highest_{mode}_rating'] = stat.get(f'chess_{mode}',{}).get('best',{}).get('rating',None)
        try:
            player_stat[f'highest_{mode}_date'] = time.strftime("%d/%m/%Y", time.localtime(int(stat[f'chess_{mode}']['best']['date'])))
        except:
            player_stat[f'highest_{mode}_date'] = None
        player_stat[f'{mode}_win'] = stat.get(f'chess_{mode}',{}).get('record',{}).get('win',None)
        player_stat[f'{mode}_draw'] = stat.get(f'chess_{mode}',{}).get('record',{}).get('draw',None)
        player_stat[f'{mode}_loss'] = stat.get(f'chess_{mode}',{}).get('record',{}).get('loss',None)
       
                        
    player_stat['highest_tactics_rating'] = stat.get('tactics',{}).get('highest',{}).get('rating',None)
    try:
        player_stat['highest_tactics_date'] = time.strftime("%d/%m/%Y", time.localtime(int(stat['tactics']['highest']['date'])))
    except:
        player_stat['highest_tactics_date']
    player_stat['highest_puzzle_rush_score'] = stat.get('puzzle_rush',{}).get('best',{}).get('score',None)

    player_stat['is_streamer'] = profile.get('is_streamer',None)
    player_stat['status'] = profile.get('status',None)
    player_stat['league'] = profile.get('league',None)

    
    all_players_stat.append(player_stat)


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [92]:
GM_players_df = pd.DataFrame(all_players_stat)

In [96]:
GM_players_df.sort_values(by='fide', ascending=False).reset_index().drop(['index'],axis=1)

,name,username,profile,user_id,title,fide,country,followers,joined,last_online,...,highest_bullet_date,bullet_win,bullet_draw,bullet_loss,highest_tactics_rating,highest_tactics_date,highest_puzzle_rush_score,is_streamer,status,league
0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,494,07/11/2015,17/09/2018,...,None,NaN,NaN,NaN,400,07/11/2015,NaN,False,premium,None
1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,360,11/08/2017,13/07/2023,...,27/09/2021,32.0,8.0,47.0,3251,15/09/2021,49.0,False,premium,Bronze
2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,199,05/11/2014,08/12/2014,...,None,NaN,NaN,NaN,400,05/11/2014,NaN,False,premium,None
3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,142482,27/08/2010,15/07/2023,...,28/03/2023,662.0,98.0,246.0,400,27/08/2010,NaN,False,premium,Elite
4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,14445,17/03/2013,16/07/2023,...,30/11/2021,713.0,65.0,203.0,3519,14/06/2023,64.0,False,premium,Crystal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,Stanojoski Zvonko,zona1,https://www.chess.com/member/Zona1,13526852,GM,NaN,North Macedonia,2,15/09/2013,20/05/2022,...,27/09/2014,13.0,1.0,7.0,400,15/09/2013,NaN,False,premium,None
1504,Dmitry Gurevich,zq1,https://www.chess.com/member/Zq1,4042329,GM,NaN,United States,127,28/09/2010,13/07/2023,...,04/04/2019,496.0,47.0,402.0,3356,19/05/2023,44.0,False,premium,Champion
1505,Alexander Zubov,zubov_alexander,https://www.chess.com/member/Zubov_Alexander,83663714,GM,NaN,Ukraine,37,21/06/2020,25/05/2022,...,None,NaN,NaN,NaN,400,21/06/2020,NaN,False,basic,None
1506,Alexander Zubov,zubov_on_youtube,https://www.chess.com/member/Zubov_On_YouTube,148025331,GM,NaN,Ukraine,5,16/07/2021,27/01/2022,...,None,NaN,NaN,NaN,400,16/07/2021,NaN,False,basic,None


In [84]:
requests.get(chessdotcom.get_player_profile('magnuscarlsen').json['player'].get('country',None)).json['name']  

TypeError: 'method' object is not subscriptable

In [90]:
requests.get(chessdotcom.get_player_profile('magnuscarlsen').json['player'].get('country',None)).json()['name']

'Norway'